In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import csv
import matplotlib.pyplot as plt
import datetime
import math
from datetime import *
import gensim
from gensim.models.doc2vec import Doc2Vec



In [2]:
train_df = pd.read_table('../data/oppo_round1_train_20180926.txt', names=['prefix', 'query_prediction', 'title', 'tag', 'label'], header=None, quoting=3)
valid_df = pd.read_table('../data/oppo_round1_vali_20180926.txt', names=['prefix', 'query_prediction', 'title', 'tag', 'label'], header=None, quoting=3)
test_df = pd.read_table('../data/oppo_round1_test_A_20180926.txt', names=['prefix', 'query_prediction', 'title', 'tag', 'label'], header=None, quoting=3)
print(valid_df.info())
print(test_df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
prefix              50000 non-null object
query_prediction    49042 non-null object
title               50000 non-null object
tag                 50000 non-null object
label               50000 non-null int64
dtypes: int64(1), object(4)
memory usage: 1.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
prefix              50000 non-null object
query_prediction    49036 non-null object
title               50000 non-null object
tag                 50000 non-null object
label               0 non-null float64
dtypes: float64(1), object(4)
memory usage: 1.9+ MB
None


In [3]:
print(len(train_df[train_df.query_prediction.isnull()]))
print(np.mean(train_df[train_df.query_prediction.isnull()]['label']))
print(np.mean(train_df['label']))
print(np.mean(valid_df[valid_df.query_prediction.isnull()]['label']))
print(np.mean(valid_df['label']))



37248
0.38815506872852235
0.3747345
0.3778705636743215
0.37626


In [2]:
train_df = pd.read_table('../data/oppo_round1_train_20180929.txt', names=['prefix', 'query_prediction', 'title', 'tag', 'label'], header=None, quoting=3)
valid_df = pd.read_table('../data/oppo_round1_vali_20180929.txt', names=['prefix', 'query_prediction', 'title', 'tag', 'label'], header=None, quoting=3)
train_df = pd.concat([train_df, valid_df])
train_df.reset_index(inplace=True)
train_df['index'] = train_df.index

test_df = pd.read_table('../data/oppo_round1_test_B_20181106.txt', names=['prefix', 'query_prediction', 'title', 'tag', 'label'], header=None, quoting=3)
# test_df = pd.read_table('../data/oppo_round1_vali_20180926.txt', names=['prefix', 'query_prediction', 'title', 'tag', 'label'], header=None, quoting=3)
print(test_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 5 columns):
prefix              250000 non-null object
query_prediction    250000 non-null object
title               250000 non-null object
tag                 250000 non-null object
label               0 non-null float64
dtypes: float64(1), object(4)
memory usage: 9.5+ MB
None


In [7]:
train_prefix_set = set(train_df['prefix'])

test_df['is_prefix_in_train'] = test_df['prefix'].map(lambda x : 1 if x in train_prefix_set else 0)
print(len(test_df[test_df.is_prefix_in_train == 1]))
print(len(set(test_df['prefix'])))
print(len(train_prefix_set))


228115
69080
162754


In [31]:
train_df = pd.read_table('../data/oppo_round1_train_20180929.txt', names=['prefix', 'query_prediction', 'title', 'tag', 'label'], header=None, quoting=3)
print(train_df.info())
print(train_df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 5 columns):
prefix              object
query_prediction    object
title               object
tag                 object
label               int64
dtypes: int64(1), object(4)
memory usage: 76.3+ MB
None
  prefix                                   query_prediction            title  \
0     小品  {"小品大全宋小宝": "0.009", "小品相亲": "0.012", "小品剧本": ...               小品   
1   1368  {"13688cc赛马会": "0.059", "13685367892": "0.124"...  HCG大于1368%2C正常吗   
2   1368  {"13688cc赛马会": "0.059", "13685367892": "0.124"...            1368年   
3     银耳  {"银耳汤的功效": "0.012", "银耳为什么不能天天吃": "0.009", "银耳...         银耳红枣汤的做法   
4   月经量少  {"月经量少喝红糖水好吗": "0.010", "月经量少该怎么调理": "0.016", ...         月经量少怎么调理   

  tag  label  
0  阅读      0  
1  健康      0  
2  百科      1  
3  菜谱      1  
4  百科      0  


In [32]:
train_df['query_prediction_dict'] = train_df['query_prediction'].astype(str).map(lambda x : eval(x))
print(train_df[['query_prediction', 'prefix']].head())




                                    query_prediction prefix
0  {"小品大全宋小宝": "0.009", "小品相亲": "0.012", "小品剧本": ...     小品
1  {"13688cc赛马会": "0.059", "13685367892": "0.124"...   1368
2  {"13688cc赛马会": "0.059", "13685367892": "0.124"...   1368
3  {"银耳汤的功效": "0.012", "银耳为什么不能天天吃": "0.009", "银耳...     银耳
4  {"月经量少喝红糖水好吗": "0.010", "月经量少该怎么调理": "0.016", ...   月经量少


In [33]:
train_df['query_prediction_values_list'] = train_df['query_prediction_dict'].map(lambda x : list(x.values()))
train_df['query_prediction_keys_list'] = train_df['query_prediction_dict'].map(lambda x : list(x.keys()))
train_df['query_prediction_len'] = train_df['query_prediction_values_list'].map(lambda x : len(x))
print(train_df[['query_prediction_values_list', 'query_prediction_len']].head())



                        query_prediction_values_list  query_prediction_len
0  [0.009, 0.012, 0.020, 0.066, 0.007, 0.010, 0.1...                    10
1  [0.059, 0.124, 0.029, 0.070, 0.022, 0.042, 0.0...                     9
2  [0.059, 0.124, 0.029, 0.070, 0.022, 0.042, 0.0...                     9
3  [0.012, 0.009, 0.050, 0.045, 0.053, 0.014, 0.0...                    10
4  [0.010, 0.016, 0.009, 0.009, 0.569, 0.016, 0.0...                    10


In [23]:
query_prediction_len_pivot_table = pd.pivot_table(train_df, index='query_prediction_len', values='label', aggfunc=len)
print(query_prediction_len_pivot_table)




                        label
query_prediction_len         
0                        6926
1                       13713
2                       14593
3                       11998
4                       11814
5                       13207
6                       12399
7                       14982
8                      128500
9                      299634
10                    1472234


In [34]:
prefix_pivot_table = pd.pivot_table(train_df, index='prefix', values='label', aggfunc=len)
prefix_pivot_table.reset_index(inplace=True)
prefix_pivot_table.rename(columns={'label' : 'prefix_number'}, inplace=True)
prefix_number_pivot_table = pd.pivot_table(prefix_pivot_table, index='prefix_number', values='prefix', aggfunc=len)
print(prefix_pivot_table.head())
print(prefix_number_pivot_table)



  prefix  prefix_number
0      #             41
1      %             33
2      &             14
3    (女人              1
4   *p++              1
               prefix
prefix_number        
1               45173
2               36675
3               17138
4               10095
5                7165
6                5136
7                3884
8                3043
9                2536
10               2123
11               1762
12               1555
13               1354
14               1215
15               1041
16                901
17                795
18                753
19                682
20                659
21                540
22                519
23                485
24                460
25                405
26                352
27                364
28                330
29                318
30                276
...               ...
2813                1
2834                1
3033                1
3158                1
3323                1
3382                

In [35]:
query_prediction_pivot_table = pd.pivot_table(train_df, index='query_prediction', values='label', aggfunc=len)
query_prediction_pivot_table.reset_index(inplace=True)
query_prediction_pivot_table.rename(columns={'label' : 'query_prediction_number'}, inplace=True)
query_prediction_number_pivot_table = pd.pivot_table(query_prediction_pivot_table, index='query_prediction_number', values='query_prediction', aggfunc=len)
print(query_prediction_pivot_table.head())
print(query_prediction_number_pivot_table)



                                    query_prediction  query_prediction_number
0  {"#*": "0.065", "#什么意思": "0.065", "#n/a是什么意思":...                       41
1              {"%2c...8一": "0.053", "%2c": "0.044"}                       33
2  {"&nbsp": "0.022", "&怎么读": "0.104", "&g90tvk75...                       14
3  {"- 千千音乐": "0.079", "- 百度": "0.274", "- 魔镜原创摄影...                        1
4  {". 新闻": "0.008", ". 新闻美女丑事": "0.012", ". 新闻 新...                        9
                         query_prediction
query_prediction_number                  
1                                   42779
2                                   35024
3                                   16782
4                                   10001
5                                    7121
6                                    5142
7                                    3873
8                                    3047
9                                    2540
10                                   2121
11                          

In [36]:
prefix_query_pivot_table = pd.pivot_table(train_df, index=['prefix', 'query_prediction'], values='label', aggfunc=len)
prefix_query_pivot_table.reset_index(inplace=True)
prefix_query_pivot_table.rename(columns={'label' : 'prefix_query_number'}, inplace=True)
prefix_query_number_query_pivot_table = pd.pivot_table(prefix_query_pivot_table, index='prefix_query_number', values='prefix', aggfunc=len)
print(prefix_query_number_query_pivot_table)


                     prefix
prefix_query_number        
1                     45173
2                     36675
3                     17138
4                     10095
5                      7165
6                      5136
7                      3884
8                      3043
9                      2536
10                     2123
11                     1762
12                     1555
13                     1354
14                     1215
15                     1041
16                      901
17                      795
18                      753
19                      682
20                      659
21                      540
22                      519
23                      485
24                      460
25                      405
26                      352
27                      364
28                      330
29                      318
30                      276
...                     ...
2813                      1
2834                      1
3033                

In [38]:
valid_df = pd.read_table('../data/oppo_round1_vali_20180929.txt', names=['prefix', 'query_prediction', 'title', 'tag', 'label'], header=None, quoting=3)
print(valid_df.info())
print(valid_df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
prefix              50000 non-null object
query_prediction    50000 non-null object
title               50000 non-null object
tag                 50000 non-null object
label               50000 non-null int64
dtypes: int64(1), object(4)
memory usage: 1.9+ MB
None
  prefix                                   query_prediction  title tag  label
0   重庆旅游  {"重庆旅游景区": "0.018", "重庆旅游攻略": "0.373", "重庆旅游景点...  皇包车旅行  应用      0
1   婆婆来了  {"婆婆来了大结局": "0.021", "婆婆来了电视剧": "0.100", "婆婆来了...   搜狐视频  应用      0
2    张国荣  {"张国荣遗体很恐怖": "0.020", "张国荣头像": "0.013", "张国荣24...    张国荣  歌手      0
3     陌陌  {"陌陌晒奶": "0.008", "陌陌下载安装": "0.009", "陌陌交友": "...     陌陌  百科      0
4     畅游  {"畅游阁": "0.322", "畅游六零年代": "0.134", "畅游书城82212...    李畅游  百科      0


In [39]:
valid_df['query_prediction'] = valid_df['query_prediction'].astype(str)
print(valid_df[['query_prediction', 'title']].head())



                                    query_prediction  title
0  {"重庆旅游景区": "0.018", "重庆旅游攻略": "0.373", "重庆旅游景点...  皇包车旅行
1  {"婆婆来了大结局": "0.021", "婆婆来了电视剧": "0.100", "婆婆来了...   搜狐视频
2  {"张国荣遗体很恐怖": "0.020", "张国荣头像": "0.013", "张国荣24...    张国荣
3  {"陌陌晒奶": "0.008", "陌陌下载安装": "0.009", "陌陌交友": "...     陌陌
4  {"畅游阁": "0.322", "畅游六零年代": "0.134", "畅游书城82212...    李畅游


In [41]:
prefix_repeat_set = set(prefix_pivot_table['prefix'][prefix_pivot_table.prefix_number > 1])
print(len(prefix_repeat_set))



111910


In [42]:
train_df['is_repeat_prefix'] = train_df['prefix'].map(lambda x : 1 if x in prefix_repeat_set else 0)
print(train_df[['prefix', 'is_repeat_prefix']].head())



  prefix  is_repeat_prefix
0     小品                 1
1   1368                 1
2   1368                 1
3     银耳                 1
4   月经量少                 1
